In [5]:
import os
import numpy as np
import mdtraj as md
import pickle
from simtk.openmm import unit
from tqdm import tqdm_notebook
from simtk.openmm import app
import matplotlib.pyplot as plt
from openeye import oechem

In [2]:
def new_positions(hybrid_positions):
    n_atoms_new = htf._topology_proposal.n_atoms_new
    hybrid_indices = [htf._new_to_hybrid_map[idx] for idx in range(n_atoms_new)]
    return hybrid_positions[hybrid_indices, :]
    
def old_positions(hybrid_positions):
    n_atoms_old = htf._topology_proposal.n_atoms_old
    hybrid_indices = [htf._old_to_hybrid_map[idx] for idx in range(n_atoms_old)]
    return hybrid_positions[hybrid_indices, :]

In [35]:
def make_traj(outdir, htf, i, phase, name, endstate, is_old=True, solvent_only=False):
    with open(os.path.join(outdir, f"{i}_{phase}_{name}_1ns_snapshots.npy"), "rb") as f:
        cache = np.load(f)
    if is_old:
        old_pos = np.zeros(shape=(100, htf._topology_proposal.old_topology.getNumAtoms(), 3))
        for j, pos in tqdm_notebook(enumerate(cache)):
            old_pos[j] = old_positions(unit.Quantity(pos, unit=unit.nanometers)).value_in_unit_system(unit.md_unit_system)
        
        traj = md.Trajectory(old_pos, md.Topology.from_openmm(htf._topology_proposal.old_topology))
    else:
        new_pos = np.zeros(shape=(100, htf._topology_proposal.new_topology.getNumAtoms(), 3))
        for j, pos in tqdm_notebook(enumerate(cache)):
            new_pos[j] = new_positions(unit.Quantity(pos, unit=unit.nanometers)).value_in_unit_system(unit.md_unit_system)
        
        traj = md.Trajectory(new_pos, md.Topology.from_openmm(htf._topology_proposal.new_topology))
        
    box_vectors = np.array([val.value_in_unit_system(unit.md_unit_system) for val in htf.hybrid_system.getDefaultPeriodicBoxVectors()])
    vectors = np.array([box_vectors]) 
        
    if solvent_only:
        traj.atom_slice(traj.topology.select("water or resname 'na\+' or resn 'cl\-'"), inplace=True)
        
    # Set unit cell vectors in traj 
    traj.unitcell_vectors = np.repeat(vectors, 100, axis=0)
    traj = traj.image_molecules()
    
    if solvent_only:
        traj.save(os.path.join(outdir, f"{i}_{phase}_{endstate}_solvent.dcd"))
        traj[0].save(os.path.join(outdir, f"{i}_{phase}_{endstate}_solvent.pdb"))
    else:
        traj.save(os.path.join(outdir, f"{i}_{phase}_{endstate}.dcd"))
        traj[0].save(os.path.join(outdir, f"{i}_{phase}_{endstate}.pdb"))
    return traj
       

## 17/8 K27A

In [36]:
outdir = "/data/chodera/zhangi/perses_benchmark/neq/17/8/"
i = 8
phase = 'complex'
endstate = 0
name = 'lys'
is_old = True

with open(os.path.join(outdir, f"{i}_{phase}_{endstate}.pickle"), "rb") as f:
    htf = pickle.load(f)
    
make_traj(outdir, htf, i, phase, name, endstate, is_old)

<ipython-input-35-cc0c8912dc55>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for j, pos in tqdm_notebook(enumerate(cache)):


0it [00:00, ?it/s]

<mdtraj.Trajectory with 100 frames, 29474 atoms, 8989 residues, and unitcells at 0x2b3f25442700>

In [37]:
outdir = "/data/chodera/zhangi/perses_benchmark/neq/17/8/"
i = 8
phase = 'complex'
endstate = 1
name = 'ala'
is_old = False

with open(os.path.join(outdir, f"{i}_{phase}_{endstate}.pickle"), "rb") as f:
    htf = pickle.load(f)
    
make_traj(outdir, htf, i, phase, name, endstate, is_old)

<ipython-input-35-cc0c8912dc55>:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for j, pos in tqdm_notebook(enumerate(cache)):


0it [00:00, ?it/s]

<mdtraj.Trajectory with 100 frames, 29462 atoms, 8989 residues, and unitcells at 0x2b3f2ec28f70>